In [1]:
import torch.nn
import os
from data.data_utils import GTZAN
from torch.utils.data import DataLoader
import torch.nn
from training.inference import get_latents

def generate_evaluation_dataset(model, dataset_name, dataloader, name, chunking=True, averaging=False, chunk_size=256):
    model.mask_ratio = 0.0
    latents, labels = get_latents(dataloader, model, chunking=chunking, averaging=averaging, chunk_size=chunk_size)
    print("Saving...")

    index = 0

    for latent, label in zip(latents, labels):
        directory = f"D:\\SongsDataset\\{dataset_name}\\latent_datasets\\{name}\\"

        directory += "full-set\\"

        os.makedirs(os.path.dirname(directory), exist_ok=True)

        if not averaging and chunking:
            for x in range(label.shape[0]):
                label_name = directory + f"{index}_label"
                latent_name = directory + f"{index}_data"
                torch.save(latent, latent_name + ".pt")
                torch.save(label, label_name + ".pt")
                index += 1
        else:
            label_name = directory + f"{index}_label"
            latent_name = directory + f"{index}_data"
            torch.save(latent, latent_name + ".pt")
            torch.save(label, label_name + ".pt")
            index += 1

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import torch.nn
from torch import nn, optim


class MLP(nn.Module):
    def __init__(self, input=128, output=10, dropout=0.1):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input, 512),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(512, output)
        )

    def forward(self, x):
        return self.model(x)


class LinearProbe(nn.Module):
    def __init__(self, input=128, output=10):
        super().__init__()
        self.model = nn.Linear(input, output)

    def forward(self, x):
        return self.model(x)


# ----- Training Loop -----
def train_model(model, train_dataloader, test_dataloader, config, device="cuda"):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    model.train()
    best_accuracy = 0.0
    patience = 0

    for epoch in range(config.num_epochs):
        total_loss = 0
        for labels, data in train_dataloader:
            # if dataset returns one-hot, convert back to integer for CrossEntropy
            if labels.ndim > 1:
                labels = labels.argmax(dim=1)

            data = data.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        scores, accuracy = evaluate_model(model, test_dataloader)

        if accuracy < best_accuracy:
            patience += 1
        else:
            patience = 0
            best_accuracy = accuracy

        if patience >= 32:
            return model

    return model


# Evaluation
def evaluate_model(model, dataloader, device="cuda"):
    model.eval()
    all_predictions = []
    all_labels = []

    for labels, data in dataloader:
        # if dataset returns one-hot, convert back to integer for CrossEntropy
        if labels.ndim > 1:
            labels = labels.argmax(dim=1)

        data = data.to(device)
        labels = labels.to(device)

        outputs = model(data)
        predicted = outputs.argmax(dim=1)

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    scores = f1_score(all_labels, all_predictions, average="macro")
    accuracy = accuracy_score(all_labels, all_predictions)

    return scores, accuracy


In [ ]:
def grid_search(latent_dataset, num_classes=10):
    latents = latent_dataset.latents
    labels = latent_dataset.labels
    lgc = local_coherence(np.array(latents), np.array(labels), k=50)
    print(f"Coherence: {lgc}")

    best_params = []
    best_accuracy = 0
    best_F1 = 0
    pb = tqdm(total=27)

    n_fold_length = len(latent_dataset)
    n_factor = int(n_fold_length / 10)
    random_indicies = np.random.permutation(n_fold_length)

    n_fold = random_indicies[:n_factor]
    other_folds = random_indicies[n_factor:]

    train_set = torch.utils.data.Subset(latent_dataset, other_folds)
    test_set = torch.utils.data.Subset(latent_dataset, n_fold)

    for model_type in ["MLP", "LinearProbe"]:
        for weight_decay in [0, 1e-4, 1e-3]:
            for learning_rate in [1e-5, 1e-4, 1e-3]:
                for batch in [64]: # Should be [64, 256]
                    for dropout in [0.25, 0.5] if model_type == "MLP" else [0]: # Should be [0.25, 0.5, 0.75]

                        model_name = f"LinearClassifier-{dataset}"
                        config = Config(
                                save_path=f"trained_models\\{model_name}\\",
                                num_epochs=1024,
                                learning_rate=learning_rate,
                                weight_decay=weight_decay,
                                num_workers=2,
                                batch_size= batch,
                                eval_batch_size=batch,
                                dtype=torch.float32
                            )


                        train_latent_dataloader = DataLoader(
                            train_set,
                            batch_size=64,
                            shuffle=True,
                        )

                        test_latent_dataloader = DataLoader(
                            test_set,
                            batch_size=64,
                            shuffle=True,
                        )

                        device = "cuda"

                        if model_type == "MLP":
                            model = MLP(128, num_classes, dropout=dropout).to(device)
                        else:
                            model = LinearProbe(128, num_classes).to(device)

                        train_model(model, train_latent_dataloader, test_latent_dataloader, config, device=device)
                        f1, accuracy = evaluate_model(model, test_latent_dataloader, device)

                        if accuracy > best_accuracy:
                            best_params = [model_type, learning_rate, weight_decay, dropout]
                            best_accuracy = accuracy
                            best_F1 = f1
                        pb.update(1)

    print(f"Best Accuracy: {best_accuracy}")
    print(f"Best F1: {best_F1}")

    return best_params

In [2]:
from data.data_utils import GTZAN
from torch.utils.data import DataLoader
import torch.nn

directory = "D:\\SongsDataset\\GTZAN\\genres_original\\"
GTZAN_dataset = GTZAN(directory)
GTZAN_dataloader = DataLoader(
    GTZAN_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=2,
    prefetch_factor=1
)

def magnatagatune(model, name):
    generate_evaluation_dataset(model, "GTZAN", GTZAN_dataloader, name, chunking=True, chunk_size=256, averaging=True)
    num_classes = 10
    directory = f"D:\\SongsDataset\\GTZAN\\latent_datasets\\{name}\\"
    latent_dataset = LatentDataset(directory + "full-set\\", num_classes=num_classes)

    grid_search()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
def process_song():


In [3]:
model = torch.load("E:\\Coding\\SongAnalyzer\\Analyzer\\src\\trained_models\\Myna-CLS-Sinusoidal\\Epoch-79.pt", weights_only=False)
model.encoding = "sinusoidal"
generate_evaluation_dataset(model, GTZAN_dataloader, "Sinusoidal-Chunking-256", chunking=True, chunk_size=256, averaging=True)

100%|██████████| 999/999 [00:30<00:00, 33.10it/s]


Saving...


In [11]:
model = torch.load("E:\\Coding\\SongAnalyzer\\Analyzer\\src\\trained_models\\Myna-CLS-ALIBI\\Epoch-79.pt", weights_only=False)
model.encoding = "2D-ALIBI"

In [12]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-256", chunking=True, chunk_size=256, averaging=True)


100%|██████████| 999/999 [00:51<00:00, 19.37it/s]


Saving...


In [13]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-512", chunking=True, chunk_size=512, averaging=True)


100%|██████████| 999/999 [00:31<00:00, 31.60it/s]


Saving...


In [14]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-1024", chunking=True, chunk_size=1024, averaging=True)


100%|██████████| 999/999 [00:29<00:00, 33.34it/s]


Saving...


In [15]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-2048", chunking=True, chunk_size=2048, averaging=True)


100%|██████████| 999/999 [00:42<00:00, 23.59it/s]


Saving...


In [16]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-4096", chunking=True, chunk_size=4096, averaging=True)


100%|██████████| 999/999 [00:20<00:00, 47.99it/s] 


Saving...


In [17]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-No-Chunking", chunking=False, chunk_size=16384)


  0%|          | 0/999 [00:00<?, ?it/s]E:\Coding\SongAnalyzer\Analyzer\src\training\inference.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data)

100%|██████████| 999/999 [01:10<00:00, 14.15it/s]


Saving...


In [3]:
from training.evaluation import local_coherence
from data.data_utils import LatentDataset
import numpy as np
import tqdm
from utils.Config import Config
from data.data_utils import *
import torch.nn
import os
from data.data_utils import GTZAN
from torch.utils.data import DataLoader
import torch.nn
from training.inference import get_latents



In [4]:
def test_dataset(dataset, model_type, learning_rate, weight_decay, dropout, batch_size, count=12):
    directory = f"D:\\SongsDataset\\GTZAN\\latent_datasets\\{dataset}\\"

    num_classes = 10
    latent_dataset = LatentDataset(directory + "full-set\\", num_classes=num_classes)

    f1s = []
    accuracies = []

    for x in tqdm(range(count)):
        n_fold_length = len(latent_dataset)
        n_factor = int(n_fold_length / 10)

        random_indicies = np.random.permutation(n_fold_length)

        n_fold = random_indicies[:n_factor]
        other_folds = random_indicies[n_factor:]

        train_set = torch.utils.data.Subset(latent_dataset, other_folds)
        test_set = torch.utils.data.Subset(latent_dataset, n_fold)

        model_name = f"LinearClassifier-{dataset}"
        config = Config(
                save_path=f"trained_models\\{model_name}\\",
                num_epochs=1024,
                learning_rate=learning_rate,
                weight_decay=weight_decay,
                num_workers=2,
                batch_size= batch_size,
                eval_batch_size=batch_size,
                dtype=torch.float32
            )


        train_latent_dataloader = DataLoader(
            train_set,
            batch_size=64,
            shuffle=True,
        )

        test_latent_dataloader = DataLoader(
            test_set,
            batch_size=64,
            shuffle=True,
        )

        if model_type == "MLP":
            model = MLP(128, num_classes, dropout=dropout).to("cuda")
        else:
            model = LinearProbe(128, num_classes).to("cuda")

        device = "cuda"

        train_model(model, train_latent_dataloader, test_latent_dataloader, config, device=device)
        f1, accuracy = evaluate_model(model, test_latent_dataloader, device)
        f1s.append(f1)
        accuracies.append(accuracy)

    mean_f1 = sum(f1s) / len(f1s)
    variance_f1 = sum([(x - mean_f1) ** 2 for x in f1s]) / len(f1s)

    mean_accuracy = sum(accuracies) / len(accuracies)
    variance_accuracy = sum([(x - mean_accuracy) ** 2 for x in accuracies]) / len(accuracies)

    return (mean_f1, variance_f1), (mean_accuracy, variance_accuracy)

In [7]:
dataset = "Sinusoidal-Chunking-256"
test_latent_dataaset(dataset)

  0%|          | 0/27 [00:00<?, ?it/s]

Local Coherence: 0.4187387387387388
MLP
0.001
0.0001
0.25
Best Accuracy: 0.8484848484848485
Best F1: 0.8354591160473512


In [19]:
# [model_type, learning_rate, weight_decay, dropout]
f1, accuracy = test_dataset("Sinusoidal-Chunking-256", "MLP", 0.001, 0.0001, 0.25, 64)
mean_f1, variance_f1 = f1
mean_accuracy, variance_accuracy = accuracy

print(f"F1\nMean: {mean_f1:.04f} | Variance: {variance_f1:.04f}")
print(f"\n")
print(f"Accuracy\nMean: {mean_f1:.04f} | Variance: {variance_f1:.04f}")

  0%|          | 0/12 [00:00<?, ?it/s]

F1
Mean: 0.7493 | Variance: 0.0004


Accuracy
Mean: 0.7493 | Variance: 0.0004


In [7]:
dataset = "ALIBI-Chunking-256"
test_latent_dataaset(dataset)

  0%|          | 0/27 [00:00<?, ?it/s]

Local Coherence: 0.4005205205205205
MLP
0.0001
0.001
0.25
Best Accuracy: 0.7575757575757576
Best F1: 0.7395081078341948


In [20]:
# [model_type, learning_rate, weight_decay, dropout]
f1, accuracy = test_dataset("ALIBI-Chunking-256", "MLP", 0.0001, 0.001, 0.25, 64)
mean_f1, variance_f1 = f1
mean_accuracy, variance_accuracy = accuracy

print(f"F1\nMean: {mean_f1:.04f} | Variance: {variance_f1:.04f}")
print(f"\n")
print(f"Accuracy\nMean: {mean_f1:.04f} | Variance: {variance_f1:.04f}")

  0%|          | 0/12 [00:00<?, ?it/s]

ValueError: Unknown format code 'd' for object of type 'float'

In [8]:
dataset = "ALIBI-Chunking-512"
test_latent_dataaset(dataset)

  0%|          | 0/27 [00:00<?, ?it/s]

Local Coherence: 0.39511511511511516
MLP
0.0001
0.001
0.25
Best Accuracy: 0.7575757575757576
Best F1: 0.746513143719026


In [22]:
# [model_type, learning_rate, weight_decay, dropout]
f1, accuracy = test_dataset("ALIBI-Chunking-512", "MLP", 0.0001, 0.001, 0.25, 64)
mean_f1, variance_f1 = f1
mean_accuracy, variance_accuracy = accuracy

print(f"F1\nMean: {mean_f1:.04f} | Variance: {variance_f1:.04f}")
print(f"\n")
print(f"Accuracy\nMean: {mean_f1:.04f} | Variance: {variance_f1:.04f}")

  0%|          | 0/12 [00:00<?, ?it/s]

F1
Mean: 0.7349 | Variance: 0.0012


Accuracy
Mean: 0.7349 | Variance: 0.0012


In [25]:
dataset = "ALIBI-Chunking-1024"
test_latent_dataaset(dataset)

  0%|          | 0/27 [00:00<?, ?it/s]

Local Coherence: 0.38308308308308314


KeyboardInterrupt: 

In [23]:
# [model_type, learning_rate, weight_decay, dropout]
f1, accuracy = test_dataset("ALIBI-Chunking-1024", "MLP", 0.0001, 0.0, 0.5, 64)
mean_f1, variance_f1 = f1
mean_accuracy, variance_accuracy = accuracy

print(f"F1\nMean: {mean_f1:.04f} | Variance: {variance_f1:.04f}")
print(f"\n")
print(f"Accuracy\nMean: {mean_f1:.04f} | Variance: {variance_f1:.04f}")

  0%|          | 0/12 [00:00<?, ?it/s]

F1
Mean: 0.7144 | Variance: 0.0020


Accuracy
Mean: 0.7144 | Variance: 0.0020


In [10]:
dataset = "ALIBI-Chunking-2048"
test_latent_dataaset(dataset)

  0%|          | 0/27 [00:00<?, ?it/s]

Local Coherence: 0.38174174174174175
MLP
0.0001
0.0001
0.25
Best Accuracy: 0.7575757575757576
Best F1: 0.7444322344322345


In [24]:
# [model_type, learning_rate, weight_decay, dropout]
f1, accuracy = test_dataset("ALIBI-Chunking-2048", "MLP", 0.0001, 0.0001, 0.25, 64)
mean_f1, variance_f1 = f1
mean_accuracy, variance_accuracy = accuracy

print(f"F1\nMean: {mean_f1:.04f} | Variance: {variance_f1:.04f}")
print(f"\n")
print(f"Accuracy\nMean: {mean_f1:.04f} | Variance: {variance_f1:.04f}")

  0%|          | 0/12 [00:00<?, ?it/s]

F1
Mean: 0.6965 | Variance: 0.0039


Accuracy
Mean: 0.6965 | Variance: 0.0039


In [5]:
dataset = "ALIBI-No-Chunking"
test_latent_dataaset(dataset)

  0%|          | 0/27 [00:00<?, ?it/s]

MLP
0.0001
0
0.25
Best Accuracy: 0.7676767676767676
Best F1: 0.7656357829210728


In [6]:
dataset = "ALIBI-No-Chunking"
f1, accuracy = test_dataset("ALIBI-No-Chunking", "MLP", 0.0001, 0, 0.25, 64)
mean_f1, variance_f1 = f1
mean_accuracy, variance_accuracy = accuracy



  0%|          | 0/12 [00:00<?, ?it/s]

ValueError: Unknown format code 'd' for object of type 'float'

In [7]:
print(f"F1\nMean: {mean_f1:.04f} | Variance: {variance_f1:.04f}")
print(f"\n")
print(f"Accuracy\nMean: {mean_f1:.04f} | Variance: {variance_f1:.04f}")

F1
Mean: 0.7201 | Variance: 0.0031


Accuracy
Mean: 0.7201 | Variance: 0.0031
